In [1]:
import joblib
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import seaborn as sns

#특수기호 제거 패키지 (krwordrank 제공)
from krwordrank.hangle import normalize 
# Token 패키지
from ckonlpy.tag import Twitter
from konlpy.tag import Hannanum
from konlpy.tag import Kkma

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC, LinearSVC
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity



## 1.과제, 지원사업 데이터 불러오기

In [2]:
comp_raw=pd.read_excel('./final_data/company_final_new_210222.xlsx')
support_raw=pd.read_excel('./final_data/support_final_new_210222.xlsx')

company=comp_raw
support=support_raw

### 1-1.소관부처(범주형 데이터) 라벨링

In [3]:
category_dict_decod={0: '과학기술정보통신부',
 1: '행정안전부',
 2: '농촌진흥청',
 3: '문화체육관광부',
 4: '중소벤처기업부',
 5: '산업통상자원부',
 6: '교육부'}
category_dict_encod={'과학기술정보통신부': 0,
 '행정안전부': 1,
 '농촌진흥청': 2,
 '문화체육관광부': 3,
 '중소벤처기업부': 4,
 '산업통상자원부': 5,
 '교육부': 6}

### 1-2.TfidfVector fit
지원 okt+hannanum kkma +   과제 okt+hannanum+kkma

In [4]:
comp_word=company['okt_han_kkma'].tolist()
sup_word=support['all_word_new'].tolist()

# 지원사업, 과제 데이터 okt, hannanum, kkma 토큰화 단어
word_list=list(set(comp_word+sup_word))

#TfidfVectorizer
tfidf = TfidfVectorizer(analyzer = 'word', ngram_range = (1,2), min_df =5, sublinear_tf = True)
tfidf.fit(word_list)

TfidfVectorizer(min_df=5, ngram_range=(1, 2), sublinear_tf=True)

In [5]:
#SVC 모델 불러오기
lisvc_model = joblib.load('./final_data/lisvc_model_0225.pkl')

LinearSVC Model  
LinearSVC(C=1000,max_iter=500, penalty='l2')

## 2.사용자로 부터 과제내용 입력받기

In [6]:
# 불용어 사전 불러오기
stopword=''
f =open("./final_data/stop_words.txt","r", encoding = 'utf-8') 
for line in f:
    stopword += line

In [7]:
'''
##입력받은 사용자의 연구내용 전처리를 위한 함수##
1. 특수기호 제거
2.Twitter, Hannanum, Kkma 토큰화 후 중복없이 단어 합치기
'''
twitter = Twitter()
hannanum = Hannanum()
kkma=Kkma() 

#토큰화 패키지 리스트
token_model = [twitter, hannanum, kkma]
#추출 형태소 (명사, 영어)
tag_list = ['Alpha', "Noun", 'F', "N", 'OL', "NNG"]

def cleansing_user(text):
    #소문자 변경 및 특수기호 제거
    clean_text = normalize(text, english=True, number=True)
    clean_text=clean_text.lower()
    
    text_nouns=[]
    for model in tqdm_notebook(token_model):
        text_pos = model.pos(clean_text)  #형태소 분석

        for word, tag in text_pos:
            #명사, 영어 추출 & 불용어 제거
            if (tag in tag_list) and (not word in stopword): 
                #단어길이 2이상
                if len(word)>1:                             
                    text_nouns.append(word)
    
    text_nouns = list(set(text_nouns))
    clean_text_token = ' '.join(text_nouns)

    return clean_text_token
            
    

C:\Users\user\anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [8]:
#Twitter 사용자 사전 추가
twitter.add_dictionary(['머신러닝','딥러닝','인공지능','빅데이터','machine learning','machinelearning',
                        'deep learning','bigdata','의과학','의공학','데이터','라이센서','핸드피스',
                        'hand piece','handpiece','sw','하이브리드','보철물','지르코니아','성형체','우회기법',
                        '고반응성','3d','o2o','5g','4세대','2d','5세대','co2','4k','2차원','3차원'],'Noun')

In [9]:
#사용자로 부터 과제 내용 입력
user_input = input("과제 내용을 입력해주세요\n")

과제 내용을 입력해주세요
사물 인터넷 IoT 기술을 적용하여 제품의 사전 점검 및 고객의 건강 관리 기능을 강화할 수 있는 IoT 서비스 기반 기술 개발 Wi Fi 기반 Smart Things 개발 IoT Module을 탑재한 개인용 온열기 제품화 척추 상하 높낮이 좌우 이동이 가능한 3D 모션 마사지 모듈 및 제어 기술 개발 구조 해석을 통한 BLDC 모터 구동 메카니즘 설계 최적화 하여 마사지 모듈 소형화 경량화 실현 생산성 향상을 통해 원가 경쟁력을 확보할 수 있는 조립 용이성 구조 도출 고 내구성 확보를 위한 최적화 구조 설계 디자인 실현 모터 동력 손실 최소화 구조 도출 공간 활용성을 높이고 편의성에 대한 소비자 니즈를 충족시킬 수 있는 리클라이닝이 가능한 프레임 구조 기술 개발 구동 메카니즘 검토를 위한 시작품 개발 및 제작 소비자 사용 편의성 확보를 위한 좌식 와식 상태별 각도 최적 설계 구현 공간 활용성 증대를 위한 리클라이닝 구동 메카니즘 구조 개발 사용자 안전 확보를 위한 리클라이닝 안전 시스템 구조 구현 생체 정보 측정 및 분석을 통하여 사용자별 건강 상태에 맞는 맞춤형 마사지를 제공하기 위한 심탄도 및 생체정보 측정 분석 기술 개발 동 잡음 영향을 최소화를 위한 아날로그 필터 디지털 필터 기술 구현 광량 최적 지능 제어 기술을 위한 광량 센싱 및 정전류 제어 기술 구현 생체 정보 측정 및 분석을 통하여 사용자별 건강 상태에 맞는 맞춤형 마사지를 제공 하기 위한 심탄도 및 생체정보 측정 분석 기술 개발 신체 상태 측정과 측정 결과 분석 기술 구현 허가 임상 환자 모집 및 처치


In [10]:
user_input=user_input.lower()  #영어소문자 변화
user_input_clean = cleansing_user(user_input)  #특수기호 제거 및 토큰화

<ipython-input-7-23db1adc564f>:21: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for model in tqdm_notebook(token_model):


In [11]:
user_input_clean=pd.Series(user_input_clean) #type변환

In [12]:
user_input_tfidf= tfidf.transform(user_input_clean) #백터화

In [13]:
pred=lisvc_model.predict(user_input_tfidf)  #모델 예측
pred_category = category_dict_decod[pred[0]]  #결과(소관부처) int에서 부처명으로 변경
print('분류된 소관부처 : ', pred_category) #소관부처 결과 출력

분류된 소관부처 :  중소벤처기업부


In [14]:
#분류된 소관부처에 해당하는 공고 Filter
support_filter = support[support['소관부처']==pred_category]

# 3.Tfidf를 사용하여 유사도 구하기

In [15]:
#word2vec 학습 데이터 불러오기
import pickle
f = open("./final_data/model_word2vec.pkl", "rb")
model = pickle.load(f)
f.close()

In [16]:
#유사도를 측정하기 위해 임시 데이터프레임 생성
similarity_df=company[['내역사업명','okt_han_kkma']]

In [17]:
# countvectorize 작업을 통해 tfidf를 실행하고 원하는 결과 값 구하기.
def tfidf_cal():
    test_mat = tfidf.fit_transform(comp_raw['okt_han_kkma'])
    test_sim = cosine_similarity(test_mat, test_mat) 
    test_sim_sorted_ind = test_sim.argsort()[:,::-1]
    similar_RnD = find_sim_RnD(comp_raw, test_sim_sorted_ind, user_input)
    return similar_RnD[['내역사업명']][1:]

    
def find_sim_RnD(df, sorted_ind, title_name, top_n=10):
    global similar_indexes
    title_Rnd = df[df['okt_han_kkma'] == title_name]
    title_index = title_Rnd.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    similar_indexes = similar_indexes.reshape(-1)
    return df.iloc[similar_indexes]

#사용자가 새로운 데이터 입력.
def append_word(word):
    global comp_raw
    global user_input
    comp_raw=comp_raw.append({'okt_han_kkma' : word } , ignore_index=True)
    user_input = comp_raw['okt_han_kkma'].iloc[-1]
    


In [18]:
#키워드 추가 함수
append_word(user_input_clean[0])
print(user_input_clean[0])

bldc 정전 심탄도 광량 지능 상하 각도 리클라이닝 맞춤형 와식 생체 smart 처치 좌식 경량화 척추 센싱 프레 원가 상태별 wi 열기 클라 임상 마사 내구성 아날로그 module 경량 용성 강화 온열기 신체 필터 디지털 탄도 things 심탄 생산성 iot 모션 높이 환자 시작 소형화 마사지 모집 편의성 메카니즘 사물 디자인 인터넷 높낮이 의성 용이성 활용성 건강 이동 모듈 라이닝 탑재한 사용자별 니즈 이닝 제품화 3d 개인용 시작품 공간 생체정보 점검 용이 프레임


In [19]:
#다시 카운터벡터라이징해주는 함수
tfidf_cal()

,내역사업명
3158,제한공모 2018년 1차 World Class 300 프로젝트 R D 지원 신청 안내
6222,2020년도 구매조건부신제품개발사업 구매연계형 과제 자유응모 2차 시행계획 공고
3092,2019년도 창업성장기술개발사업 혁신형 창업과제 기술기반 창업형 제2차 시행계획 공고
6804,특구 연구성과 사업화
5985,2020년도 기술규제 해결형 기술개발사업 시행계획 2차 공고 1단계
2901,고용위기기업부설연구소R D전문인력활용지원 R D
2979,2018년 팁스 TIPS 프로그램 민간투자주도형 기술창업지원 창업팀 지원계획 수정 ...
335,2020년도 창업성장기술개발사업 전략형 창업과제 4IR 제1차 시행계획 공고
988,2020년도 창업성장기술개발사업 전략형 창업과제 4IR 제1차 시행계획 공고


# 4.Word2vec를 사용하여 정확도 올리기

In [20]:
# 사용자가 입력한 각 단어의 유사도를 측정해서 dict형태로 저장.
def similar_data(sim_index):
    sim = []
    sim = model.wv.most_similar(positive=[sim_index],topn = 50)
    sim_ret = dict(sim)
    return sim_ret

#위에서 구한 사용자가 입력한 각 단어의 유사도들(300개씩)과 사용자가 입력한 데이터와 가장 유사도가 비슷한 작년 데이터를 비교해서 삭제
def standard_data(standard):
    list_standard = standard.split(" ") #사용자 입력 데이터를 리스트화.
    most_list = {} #입력 데이터의 유사도가 높은 300개 단어들을 dict형태로 저장.
    kk = [] #계속 새롭게 들어갈 데이터
    while True:
        try:
            for i in list_standard:
                kk = dict(model.wv.most_similar(positive=[i],topn = 50))
                most_list.update(kk)
                list_standard.remove(i)
                if list_standard == []:
                    return most_list
        except:
            list_standard.remove(i)
            break
    return most_list


In [21]:
double_new_sim_data = [] #유사하다고 뽑힌 데이터 중 원래의 데이터의 유사성 랭크 300위 안에 드는 단어들만 모은 리스트
okt_han_krwrhan = comp_raw['okt_han_kkma'].iloc[similar_indexes[1]].split(" ")  #가장 유사하다고 뽑힌 데이터 list화
for i in standard_data(comp_raw['okt_han_kkma'].iloc[similar_indexes[0]]).keys():
    for j in okt_han_krwrhan:
        if similar_data(i).get(j): # i => 사용자 입력 데이터. 즉, 사용자 입력 데이터를 갖고 구한 유사도에 j(작년 기준 데이터)이 
            #있나 확인.
            double_new_sim_data.append(j)  

In [22]:
double_new_sim_data2 = list(set(double_new_sim_data)) #중복값 제거.
StrA = " ".join(double_new_sim_data2)
standard_test = StrA+ " " +comp_raw['okt_han_kkma'].iloc[similar_indexes[0]] #유사도 300개안에 드는 단어들(StrA) + 사용자 입력 데이터

In [23]:
#모인 자료를 가지고 올해 사업리스트에서 추천해주기. 
def tfidf_cal():
    test_mat_2 = tfidf.fit_transform(support_filter['all_word_new'])
    test_sim_2 = cosine_similarity(test_mat_2, test_mat_2) 
    test_sim_sorted_ind_2 = test_sim_2.argsort()[:,::-1]
    similar_RnD_2 = find_sim_RnD_support(support_filter, test_sim_sorted_ind_2, user_input_2)
    return similar_RnD_2[['지원사업명','상세 URL']][1:]
def find_sim_RnD_support(df, sorted_ind, title_name, top_n=10):
    title_Rnd = df[df['all_word_new'] == title_name]
    title_index = title_Rnd.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    similar_indexes = similar_indexes.reshape(-1)
    return df.iloc[similar_indexes]
def append_word_support(word):
    global support_filter
    global user_input_2
    support_filter=support_filter.append({'all_word_new' : word } , ignore_index=True)
    user_input_2 = support_filter['all_word_new'].iloc[-1]


In [25]:
append_word_support(standard_test)
tfidf_cal()

,지원사업명,상세 URL
153,2020년 스마트상점 기술보급 사업 스마트기술 보유기업 추가 모집 공고,http://www.bizinfo.go.kr/see/seea/selectSEEA14...
17,2021년 혁신조달 연계형 정부 R D 사업 공공부문 공동 수요조사 공고,http://www.bizinfo.go.kr/see/seea/selectSEEA14...
352,2020년 업종별 특화 뿌리업종 스마트공장 구축 지원사업 공고,http://www.bizinfo.go.kr/see/seea/selectSEEA14...
421,스마트센서 선도프로젝트 기술개발 사업 2020년 중소기업 스마트제조혁신 지원사업 통...,http://www.bizinfo.go.kr/see/seea/selectSEEA14...
539,혁신분야 창업패키지 3대 신산업분야 2020년 정부 창업지원사업 통합 공고,http://www.bizinfo.go.kr/see/seea/selectSEEA14...
116,동남아시아 싱가포르 진출 지원을 위한 언택트 기업설명회 IR 참가기업 모집 공고,http://www.bizinfo.go.kr/see/seea/selectSEEA14...
400,2020년 1차 중소기업 R D기획지원사업시행계획 공고,http://www.bizinfo.go.kr/see/seea/selectSEEA14...
227,2020년 스마트상점 기술보급 사업 스마트상점 모델샵 입점 기업 모집 공고,http://www.bizinfo.go.kr/see/seea/selectSEEA14...
29,2021년 선도형 디지털 클러스터 지원사업 공고 2021년 스마트공장 보급 확산사업,http://www.bizinfo.go.kr/see/seea/selectSEEA14...
